In [1]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from scipy import integrate

import models
import payoffs
import plotting
import selection_functions

In [2]:
%matplotlib inline
plt.style.use("ggplot")

# Basic simulation

In [3]:
integrate.solve_ivp?

In [3]:
# random initial condition
prng = np.random.RandomState(42)
number_of_genotypes = 4
initial_offspring_share, = prng.dirichlet(np.ones(number_of_genotypes), 1)
y0 = initial_offspring_share

# define the selection functions
d1, d3 = 1.5, 1
UGA = lambda x_A: selection_functions.kirkpatrick_selection(x_A, d1)
UgA = lambda x_A: selection_functions.kirkpatrick_selection(x_A, d3)

# define the payoffs
payoff_kernel = payoffs.prisoners_dilemma_payoffs(prng)

def f(t, y):
    W = models.generalized_sexual_selection(y, UGA, UgA, payoff_kernel)
    y_dot = models.offspring_genotypes_evolution(W, y)
    return y_dot

result = integrate.solve_ivp(f, t_span=(0, 10), y0=y0, rtol=1e-9, atol=1e-12, vectorized=True)

In [4]:
result

  message: 'The solver successfully reached the interval end.'
     nfev: 530
     njev: 0
      nlu: 0
      sol: None
   status: 0
  success: True
        t: array([  0.00000000e+00,   7.73035243e-03,   8.50338767e-02,
         1.80589388e-01,   2.77094581e-01,   3.74725993e-01,
         4.73415427e-01,   5.73101566e-01,   6.73726179e-01,
         7.75234035e-01,   8.77572804e-01,   9.80692986e-01,
         1.08454925e+00,   1.18912246e+00,   1.29436792e+00,
         1.40024417e+00,   1.50671259e+00,   1.61373734e+00,
         1.72128527e+00,   1.82932584e+00,   1.93783106e+00,
         2.04677537e+00,   2.15613552e+00,   2.26589048e+00,
         2.37602129e+00,   2.48651099e+00,   2.59734446e+00,
         2.70850831e+00,   2.81999079e+00,   2.93178165e+00,
         3.04387207e+00,   3.15625453e+00,   3.26892274e+00,
         3.38187156e+00,   3.49509690e+00,   3.60859565e+00,
         3.72236565e+00,   3.83640559e+00,   3.95071495e+00,
         4.06529401e+00,   4.18014374e+00,   4.

# Interactive simulation

In [5]:
# sliders used to control the initial condition
x1_slider = widgets.FloatSlider(value=0.25, min=0.0, max=1.0, step=0.01, description=r"$x_1$")
x2_slider = widgets.FloatSlider(value=0.25, min=0.0, max=1.0, step=0.01, description=r"$x_2$")
x3_slider = widgets.FloatSlider(value=0.25, min=0.0, max=1.0, step=0.01, description=r"$x_3$")

# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=10, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=8, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=6, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=4, min=0, max=100, step=0.1, description=r"$S$")

# sliders used to control the metabolic costs
M_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$M_G$")
m_slider = widgets.FloatSlider(value=0, min=0, max=100, step=0.1, description=r"$m_g$")

# slider used to control which selection function is being used
U_slider = widgets.Dropdown(options=["kirkpatrick", "seger", "wright-bergstrom"], index=0, description=r"$U_{\gamma(j)A}$")

# slider that controls the parameters of the selection function
d1_slider = widgets.FloatSlider(value=1, min=0.0, max=10, step=0.05, description=r"$d_1$")
d3_slider = widgets.FloatSlider(value=1, min=0.0, max=10, step=0.05, description=r"$d_3$")

# slider used to control the mutation rate
e_slider = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, step=1e-3, description=r"$\epsilon$", readout_format=".3f")

# slider that controls max simulation time
max_time_slider = widgets.IntSlider(value=25, min=1, max=500, description=r"$\max t$")

w = widgets.interactive(plotting.plot_generalized_sexual_selection, x1=x1_slider, x2=x2_slider, x3=x3_slider,
                        selection_function=U_slider, d1=d1_slider, d3=d3_slider, 
                        T=T_slider, R=R_slider, P=P_slider, S=S_slider,
                        M=M_slider, m=m_slider, epsilon=e_slider,
                        max_time=max_time_slider)
display(w)

A Jupyter Widget

In [8]:
# can get access to the solution!
(solution, optimize_result) = w.result